In [16]:
import pandas as pd
import numpy as np

# Generación de datos aleatorios

In [17]:
df = pd.read_csv("datos_arreglados.csv", encoding="latin1")
df = df[df["ID"] > 60] #esto pues las primeras filas estaban falladas

Fijamos la semilla pero se puede eliminar para generar muestras diferentes y no siempre la misma.

In [18]:
np.random.seed(42)

Ahora, fijamos el número de datos que queremos crear

In [19]:
n_sint = 2000

Creamos un dataframe donde tendremos los nuevos datos, estos datos serán generados con diferentes combinaciones de "categorías"

In [20]:
df_sint = pd.DataFrame()

# 2.1. ID nuevo (que no coincida con el original)
df_sint["ID"] = range(1, n_sint + 1)

Para generar datos más realistas, los tiempos también se generaran al azar

In [21]:
df.columns = df.columns.str.strip()

# Convertimos a datetime las columnas de tiempo
for col in ["registration_started", "registration_finished"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Duración real en segundos
duracion_real = (
    df["registration_finished"] - df["registration_started"]
).dt.total_seconds().dropna()

# ----------------- TIEMPOS SINTÉTICOS -----------------
start_base = df["registration_started"].dropna().sample(
    n_sint, replace=True
).reset_index(drop=True)

dur_base = duracion_real.sample(n_sint, replace=True).reset_index(drop=True)

jitter_start = np.random.normal(loc=0, scale=600, size=n_sint)  # ±10 min
jitter_dur   = np.random.normal(loc=0, scale=30,  size=n_sint)  # ±30 s

dur_min = duracion_real.min()
dur_max = duracion_real.max()

dur_sint = dur_base + jitter_dur
dur_sint = np.clip(
    dur_sint,
    a_min=max(1, dur_min * 0.5),
    a_max=dur_max * 1.5
)

start_sint  = start_base + pd.to_timedelta(jitter_start, unit="s")
finish_sint = start_sint + pd.to_timedelta(dur_sint, unit="s")

df_sint["registration_started"]  = start_sint
df_sint["registration_finished"] = finish_sint

C:\Users\tatyf\AppData\Local\Temp\ipykernel_26256\3785665908.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\tatyf\AppData\Local\Temp\ipykernel_26256\3785665908.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


Hay varias formas de crear datos nuevos, 

In [ ]:
# Quitamos columnas ya tratadas
cols_excluir = ["ID", "registration_started", "registration_finished"]
otras_cols = [c for c in df.columns if c not in cols_excluir]

for col in otras_cols:
    col_data = df[col].dropna()

    if col_data.empty:
        # si la columna está vacía, llenamos con NaN
        df_sint[col] = [np.nan] * n_sint
        continue

    # Todas las categorías posibles y sin distribución específica
    categorias = col_data.unique()
    df_sint[col] = np.random.choice(categorias, size=n_sint)

    # si queremos la distribución de frecuencias de los valores observados
    # vc = col_data.value_counts(normalize=True)
    # categorias = vc.index.to_list()
    # probs      = vc.to_numpy()

    # podríamos hacer un sampleo categórico según la distribución original
    # df_sint[col] = np.random.choice(categorias, size=n_sint, p=probs)


    ########################################################
    # FALTA AGREGAR RESTRICCIONES COMO DE CUÁNTAS RESPUESTAS PUEDE TENER UN ESTUDIANTE EN EL FORMULARIO

# Guardar el csv y que coincidan las columnas
df_sint = df_sint[df.columns]
df_sint.to_csv("datos_nuevos.csv", index=False)
